In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
print("termino fase uno")


In [ ]:
# Configuración del navegador
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# URL de la página de LinkedIn
url = "https://www.linkedin.com/jobs/search/?currentJobId=3903380739&distance=25&geoId=100876405&keywords=analista%20de%20datos&origin=JOBS_HOME_KEYWORD_HISTORY&refresh=true"

# Navegar a la página de LinkedIn
driver.get(url)

# Esperar a que la página cargue
time.sleep(5)  # Ajusta el tiempo según la velocidad de tu conexión

# Inicializar el contador de trabajos
contador_trabajos = 0

# Realizar el desplazamiento de la página para cargar más trabajos
while True:
    # Simular el desplazamiento hacia abajo
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)  # Esperar un poco para que se carguen los nuevos elementos
    
    job_elements = driver.find_elements(By.CSS_SELECTOR, "div.base-card")
    job_ids = [job.get_attribute("data-entity-urn").split(":")[3] for job in job_elements]
    trabajos_antes=len(job_ids)
    
    # Verificar si hay un botón "Ver más empleos" visible
    try:
        ver_mas_empleos_button = driver.find_element(By.XPATH, "//button[@aria-label='Ver más empleos']")
    except NoSuchElementException:
        break  # Salir del bucle si no hay más botones "Ver más empleos"

    # Hacer clic en el botón "Ver más empleos"
    try:
        ver_mas_empleos_button.click()
    except ElementNotInteractableException:
        # Si no se puede hacer clic, hacer más desplazamiento hacia abajo y luego continuar
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Esperar un poco después de desplazarse hacia abajo
        continue

    time.sleep(2)  # Esperar un poco después de hacer clic para que se carguen los nuevos trabajos

# Obtener los IDs de los trabajos
    job_elements = driver.find_elements(By.CSS_SELECTOR, "div.base-card")
    job_ids = [job.get_attribute("data-entity-urn").split(":")[3] for job in job_elements]
    trabajos_despues=len(job_ids)
    if trabajos_despues <= trabajos_antes:
        break 
    

# Imprimir los IDs de los trabajos
# for job_id in job_ids:
    # contador_trabajos+=1
    # print(contador_trabajos,":",job_id)

# Cerrar el navegador
driver.quit()
print("termino fase dos")

In [ ]:
# Inicializar una lista vacía para almacenar la información de los trabajos
job_list = []

# Recorrer la lista de IDs de trabajos y obtener cada URL
for job_id in job_ids:
    # Construir la URL de cada trabajo usando el ID del trabajo
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    
    # Enviar una solicitud GET a la URL del trabajo y analizar la respuesta
    job_response = requests.get(job_url)
    time.sleep(2)
    
    # Imprimir el código de estado de la respuesta para depuración
    #print(job_response.status_code, "con el id:", job_id, job_url)
    
    # Parsear la respuesta con BeautifulSoup
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    
    # Crear un diccionario para almacenar los detalles del trabajo
    job_post = {}
    
    # Extraer y almacenar el título del trabajo
    try:
        job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title"}).text.strip()
    except:
        job_post["job_title"] = None
    
    # Extraer y almacenar el nombre de la empresa
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link"}).text.strip()
    except:
        job_post["company_name"] = None
    
    # Extraer y almacenar el tiempo publicado
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text"}).text.strip()
    except:
        job_post["time_posted"] = None
    
    # Extraer y almacenar el número de solicitantes
    try:
        job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption"}).text.strip()
    except:
        job_post["num_applicants"] = None

    # Extraer y almacenar la descripción del trabajo
    try:
        description_div = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if not description_div:
            description_div = job_soup.find("section", {"class": "show-more-less-html"})
        if description_div:
            job_post["job_description"] = description_div.get_text(separator="\n").strip()
        else:
            job_post["job_description"] = None
    except:
        job_post["job_description"] = None

    # Agregar los detalles del trabajo a la lista de trabajos
    job_list.append(job_post)

# Verificar si la lista contiene todos los datos deseados


In [ ]:
for job in job_list:
    print(job)
jobs_df = pd.DataFrame(job_list)
jobs_df

In [ ]:
#Save data to CSV file
jobs_df.to_csv('Toronto_Python_Developer.csv', index = False)